In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.combine import SMOTETomek

from sklearn.svm import SVM
from sklearn.linear_model import LogisticRegression
from sklearn.classifier import RandomForest

Feature Extraction - Title


(1) title length by the number of tokens

(2) title case (all uppercase?)

(3) title article (include '^the'?)

(4) title 1st person (include 'I/Me/My'?)

(5) Special Symbols

In [ ]:
## ongoing

class TitlelengthTransformer(BaseEstimator, TransformerMixin):
  def fit(self, X, y = None):
    return self

  def transform(self, X):
    return np.array([[len(title)] for title in X]).astype(int).reshape(-1, 1)

class TitlecaseTransformer(BaseEstimator, TransformerMixin):
  def fit(self, X, y = None):
    return self

  def transform(self, X):
    return np.array([[int(title.upper())] for title in X]).astype(int).reshape(-1, 1)

class TitlearticleTransformer(BaseEstimator, TransformerMixin):
  def fit(self, X, y = None):
    return self

  def transform(self, X):
    return np.array([[int('the' in title)] for title in X]).astye(int).reshape(-1, 1)

class TitlefirstpersonTransformer(BaseEstimator, TransformerMixin):
  import re

  def fit(self, X, y = None):
    return self

  def transform(self, X):

    I = [[re.match('^I', title) ] for title in X]
    Me = [[re.findall('\\bme\\b', title, flags = re.I)] for title in X]
    My = [[re.findall('\\bmy\\b', title, flags = re.I)] for title in X]

    return np.array([any(x, y, z) for x in I for y in Me for z in My]).astype(int).reshape(-1, 1)


class SpecialSymbolTransformer(BaseEstimator, TransformerMixin):
  def fit(self, X, y = none):
    return X

  def transform(self, X, symbols):
    import re
    symbols = '|'.join(symbols)
    pattern = re.compile(f'\\b{symbols}\\b')
    return np.array([[bool(pattern.search(title))] for title in X])

SpecialSymbolTransformer Test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.chdir('./drive/MyDrive/a1')
os.listdir()

['nlp2-a1.train',
 'nlp2-a1.test',
 'PoKi+A Large Dataset of Poems by Children.pdf',
 'special_symbols.txt']

In [ ]:
symbols = []
with open('special_symbols.txt', mode = 'r', encoding = 'utf8') as f:
  for symbol in f.readlines():
    symbols.append(symbol.strip())

In [ ]:
symbols

['\\!',
 '\\#',
 '\\$',
 '\\%',
 '\\&',
 "\\'",
 '\\(',
 '\\)',
 '\\*',
 '\\+',
 '\\,',
 '\\-',
 '\\.',
 '\\/',
 '\\:',
 '\\;',
 '\\<',
 '\\=',
 '\\>',
 '\\?',
 '\\@',
 '\\[',
 '\\\\',
 '\\]',
 '\\^',
 '\\`',
 '\\{',
 '\\|',
 '\\}',
 '\\~',
 '\\\x81',
 '\\\x8d',
 '\\¡',
 '\\¢',
 '\\¤',
 '\\¦',
 '\\§',
 '\\©',
 '\\\xad',
 '\\¯',
 '\\°',
 '\\±',
 '\\´',
 '\\¶',
 '\\¸',
 '\\»',
 '\\¿',
 '\\˜',
 '\\–',
 '\\—',
 '\\‘',
 '\\’',
 '\\“',
 '\\”',
 '\\€',
 '\\™']

In [ ]:
symbols = '|'.join(symbols)

In [ ]:
import re
pattern = re.compile(f'\\b{symbols}\\b')

need to escape special symbols when including them as re.compile pattern


So Class itself does not have an error, I should take care when reading the special symbol file or else escape the letters in the first time when I make special symbol file

I have escaped special symbol letters in the first place(25.06.21. 20:26)

## Pipelining

define sampler, scaler, feature extractor, estimator (some with param_grid)

In [ ]:
# define sampler, sacler, decomposition method, feature extractor, estimator
sampler = SMOTETomek()
scaler = StandardScaler()
reduce_dim = PCA()
feature_extractor = FeatueUnion(('#', 어쩌구), ...)
estimator_set1 = [LogisticRegression(), SVM()]
estimator_set2 = [RandomForest()]
estimator_set3 = [Perceptron(), MLPClassifier()]


# set parameter options
Options_n_components = [2, 5, 10, 15, 20]
Options_kernel = ['rbf', '# 어쩌구', '# 저쩌구']
Options_C = [1, 10, 100, 1000]

# set parameter grid
param_grid = [{'reduce_dim__n_components': [2, 5, 10, 15, 20],
               'estimator': estimator_set1,
               'estimator__kernel' = Options_kernel,
              'estimator__C': Options_C
               },
              {'reduce_dim__n_components': [2, 5, 10, 15, 20],
               'estimator': estimator_set2,
               'estimator__n_root': [3, 1, 2, 6],
              'estimator__C': Options_C
               },
              {'reduce_dim__n_components': [2, 5, 10, 15, 20],
               'estimator': estimator_set3,
              'estimator__C': Options_C
               },
              ]

instantiate a pipeline

In [ ]:
pipe = Pipeline([
    ('sampler', SMOTETomek()),
    ('scaler', StandardScaler()),
    ('reduce_dim', reduce_dim),
    ('estimator', 'passthrough')
])

instantiate a Cross Validation grid and fit

In [ ]:
grid = GridSearchCV(pipe,
                    param_grid)
grid.fit(#X_train, #y_train)